# File to obtain Galaxy data needed
need:
- coordinates
- group number
- stellar mass
- luminosities
- radii?

- over timescale

## Some important constants

In [1]:
sim = 'TNG300-1'
TNG300_1_boxsize = 	302.6 #Mpc
scratchDataDirc = '/scratch/poulin.al/lopsided/data'

In [6]:
from myproject.utilities import iapi_TNG, Subhalo, GalaxyGroup, ListGalaxyGroup
import os
import h5py as h5
import numpy as np

In [ ]:
baseUrl = 'http://www.tng-project.org/api/'
r=iapi_TNG.get(baseUrl)
print(r)
#check the properties of the simulation you have selected
simUrl = baseUrl+sim
print(simUrl) 
simdata : dict[str, object] = iapi_TNG.get(simUrl)
print(simdata['description'])
simdata.keys()

## Now get

### subhalo data

In [ ]:
if not os.path.exists(scratchDataDirc + 'catalogs'):
    os.makedirs(scratchDataDirc + 'catalogs')
    print(f'created directory: {scratchDataDirc} "catalogs"')
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloFlag'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloFlag')
    print(f'created directory: {scratchDataDirc} "/catalogs/SubhaloFlag"')
flag=iapi_TNG.getSubhaloField('SubhaloFlag',simulation=sim,fileName=scratchDataDirc+'catalogs/SubhaloFlag',rewriteFile=0)

In [ ]:
#let's fetch a field that will tell us about the mass of the galaxy
#SubhaloMassType gives the total mass of all bound particles, separated by particle type
mass=iapi_TNG.getSubhaloField('SubhaloMassType',simulation=sim,fileName=scratchDataDirc+'catalogs/MassType',rewriteFile=0)
print(mass.shape)

#Pull the stellar mass: 
print(mass[:,3])
stellar_mass=mass[:,4]
print(stellar_mass)

h = simdata.get('hubble')
print(h)
stellar_mass=stellar_mass*10**10/h #convert to one solar masses

In [ ]:
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloGrNr'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloGrNr')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloGrNr"')
subhaloGroupNum = iapi_TNG.getSubhaloField('SubhaloGrNr',simulation = sim,fileName=scratchDataDirc+'catalogs/SubhaloGrNr',rewriteFile=0)

In [ ]:
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloPos'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloPos')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloPos"')
subhaloPos = iapi_TNG.getSubhaloField('SubhaloPos',simulation = sim,fileName=scratchDataDirc+'catalogs/SubhaloPos',rewriteFile=0)

In [ ]:
if not os.path.exists(scratchDataDirc + 'catalogs/subhaloHalfmassRad'):
    os.makedirs(scratchDataDirc + 'catalogs/subhaloHalfmassRad')
    print(f'created directory: {scratchDataDirc} "catalogs/subhaloHalfmassRad"')
subhaloHalfmassRad = iapi_TNG.getSubhaloField('SubhaloHalfmassRad',simulation = sim,fileName=scratchDataDirc+'catalogs/subhaloHalfmassRad',rewriteFile=0)

In [ ]:
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloVmaxRad'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloVmaxRad')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloVmaxRad"')
SubhaloVmaxRad = iapi_TNG.getSubhaloField('SubhaloVmaxRad',simulation = sim,fileName=scratchDataDirc+'catalogs/SubhaloVmaxRad',rewriteFile=0)

In [ ]:
if not os.path.exists(scratchDataDirc + 'catalogs/SubhaloStellarPhotometrics'):
    os.makedirs(scratchDataDirc + 'catalogs/SubhaloStellarPhotometrics')
    print(f'created directory: {scratchDataDirc} "catalogs/SubhaloStellarPhotometrics"')
SubhaloStellarPhotometrics = iapi_TNG.getSubhaloField('SubhaloStellarPhotometrics',simulation = sim,fileName=scratchDataDirc+'catalogs/SubhaloStellarPhotometrics',rewriteFile=0)

### Group data

In [ ]:
if not os.path.exists(scratchDataDirc + 'catalogs/GroupCM'):
    os.makedirs(scratchDataDirc + 'catalogs/GroupCM')
    print(f'created directory: {scratchDataDirc} "catalogs/GroupCM"')
groupCM = iapi_TNG.getSubhaloField('GroupCM',simulation = sim,fileName=scratchDataDirc+'catalogs/GroupCM',rewriteFile=0)

In [ ]:
if not os.path.exists(scratchDataDirc + 'catalogs/GroupMCrit200'):
    os.makedirs(scratchDataDirc + 'catalogs/GroupMCrit200')
    print(f'created directory: {scratchDataDirc} "catalogs/GroupMCrit200"')
groupMCrit200 = iapi_TNG.getHaloField('Group_M_Crit200',simulation = sim,fileName=scratchDataDirc+'catalogs/GroupMCrit200',rewriteFile=0)

In [ ]:
if not os.path.exists(scratchDataDirc + 'catalogs/GroupPos'):
    os.makedirs(scratchDataDirc + 'catalogs/GroupPos')
    print(f'created directory: {scratchDataDirc} "catalogs/GroupPos"')
groupPos = iapi_TNG.getHaloField('GroupPos',simulation = sim,fileName=scratchDataDirc+'catalogs/GroupPos',rewriteFile=0)

## transform into custom classes

In [ ]:
list_of_galaxy_groups : ListGalaxyGroup = ListGalaxyGroup()
unique_group_numbers = np.unique(subhaloGroupNum)
for group_num in unique_group_numbers:
    indices = np.where(subhaloGroupNum == group_num)[0]
    member_subhalos = []
    for idx in indices:
        member_subhalos.append(Subhalo(idx, flag=flag[idx], mass=mass[idx], stellarMass=stellar_mass[idx], groupNumber=subhaloGroupNum[idx], position=subhaloPos[idx], halfMassRad=subhaloHalfmassRad[idx], vmaxRadius=SubhaloVmaxRad[idx], luminosities=SubhaloStellarPhotometrics[idx]))
    group_center_of_mass = groupCM[group_num]
    group_mass_crit200 = groupMCrit200[group_num]
    group_position = groupPos[group_num]
    galaxy_group = GalaxyGroup(groupNumber=group_num, centerOfMass=group_center_of_mass, massCrit200=group_mass_crit200, position=group_position, memberSubhalos=member_subhalos)
    list_of_galaxy_groups.addGalaxyGroup(galaxy_group)
    

## Tweak data
Filter the galaxies
- remove non cosmlogoical in origin (subhaloflag = 0)
- only mass greater than 13 solar mass
- remove groups where no single central (i.e. central pos does not match cm pos)

Correct data
- use the subhalo magnitudes from Nelson et al. (2018)
to assign luminosities to the galaxies because these include the effects
of dust obscuration and better approximate what we'd see in the real
universe (e.g., the Sloan Digital Sky Survey)
- correct the stellar masses of the galaxies following
Appendix A in Pillepich et al. (2018) because the stellar masses 
aren't as well converged in TNG300 as in the benchmark TNG100 simulation
- correct the stellar masses of the galaxies following
Appendix A in Pillepich et al. (2018) because the stellar masses 
aren't as well converged in TNG300 as in the benchmark TNG100 simulation

## save into hdf5 file
Save the data we've collected (stellar mass, group number, flag, etc) into an HDF5 file for easy access and analysis later.

In [ ]:
output_filename = scratchDataDirc + f'/galaxy_data_{sim}.h5'
with h5.File(output_filename, 'w') as f:
    list_of_galaxy_groups.save_to_hdf5(f)
print(f'Saved galaxy data to {output_filename}')